In [1]:
%load_ext autoreload
%autoreload 2
import torch
from PIL import Image
from pathlib import Path
import numpy as np

from robopose.config import LOCAL_DATA_DIR
from robopose.datasets.datasets_cfg import make_scene_dataset
from robopose.rendering.bullet_scene_renderer import BulletSceneRenderer
from robopose.visualization.singleview_articulated import make_singleview_prediction_plots
from robopose.visualization.bokeh_utils import save_image_figure

import os
from tqdm import tqdm_notebook as tqdm
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()

from robopose.utils.tensor_collection import PandasTensorCollection
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

Setting OMP and MKL num threads to 1.
Project directory :/home/dario/Documents/RL_Git/robopose
pybullet build time: Dec  4 2023 09:43:03


Loading BokehJS ...

In [5]:
# Pick Robot
renderer = BulletSceneRenderer('irb120')

Loaded EGL 1.5 after reload.


EGL device choice: 0 of 3 (from EGL_VISIBLE_DEVICES)


GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3070/PCIe/SSE2
GL_VERSION=4.6.0 NVIDIA 525.147.05
GL_SHADING_LANGUAGE_VERSION=4.60 NVIDIA
Version = 4.6.0 NVIDIA 525.147.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3070/PCIe/SSE2


In [6]:
# Pick inference result_id and dataset

dataset = 'dream.abb.synt.dr.train'

scene_ds = make_scene_dataset(dataset)
#print(f"Frame index: \n {scene_ds.frame_index.iloc[0]}")

[Memory]91.0s, 1.5min   : Loading build_frame_index...


In [24]:
# Generate visualization of the robot
# Pred :PandasTensorCollection(
#     poses: torch.Size([1, 4, 4]) torch.float32 cpu,
#     K: torch.Size([1, 3, 3]) torch.float32 cpu,
#     joints: torch.Size([1, 7]) torch.float32 cpu,
#     K_crop: torch.Size([1, 3, 3]) torch.float32 cpu,
#     keypoints_2d: torch.Size([1, 8, 2]) torch.float32 cpu,
#     TCO_keypoints_3d: torch.Size([1, 8, 3]) torch.float32 cpu,
# ----------------------------------------
#     infos:
#    label  score  scene_id  view_id  batch_im_id  \
# 0  iiwa7    1.0     10887    10887           10   

#                                          joint_names  
# 0  [iiwa_joint_1, iiwa_joint_2, iiwa_joint_3, iiw...  
# )

data = {
    'label': ['iiwa7'],
    'score': [1.0],
    'scene_id': [10887],
    'view_id': [10887],
    'batch_im_id': [10],
    'joint_names': [['iiwa_joint_1', 'iiwa_joint_2', 'iiwa_joint_3', 'iiwa_joint_4', 'iiwa_joint_5',
                     'iiwa_joint_6', 'iiwa_joint_7']]
}

infos_data = {'label': ['irb120'], 'score': [1.0], 'scene_id': [2], 'view_id': [2], 'batch_im_id': [10],
              'joint_names': [['link_1', 'link_2', 'link_3','link_3', 'link_4', 'link_5','link_6',]]}
infos_df = pd.DataFrame(infos_data)
pred = PandasTensorCollection(infos=infos_df, poses=torch.tensor([[[-0.0629, -0.9951, -0.0757, -0.0088],
         [-0.5,  0.0, -1,  1],
         [ 1, 0, -0.4376,  1.5],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]]), K=torch.tensor([[[320.   ,0. ,320.],[  0. ,320. ,240.], [  0.   ,0.  , 1.]]] ),
                                          joints=torch.ones(1, 7), K_crop=torch.ones(1, 3, 3),
                                          keypoints_2d=torch.rand(1, 8, 2), TCO_keypoints_3d=torch.tensor([[-0.06293759, -0.9951439 , -0.07568064, -0.0087784 ],
                                                                                                    [-0.43368077,  0.09556758, -0.8959843 ,  0.21344575],
                                                                                                    [ 0.8988658 , -0.02356984, -0.43758953,  1.4445719 ],
                                                                                                    [ 0.        ,  0.        ,  0.        ,  1.        ]]))
figure = make_singleview_prediction_plots(scene_ds, renderer, pred)
print(figure)
show(figure['pred_overlay'])

{'input_im': Figure(id='5051', ...), 'pred_rendered': Figure(id='5094', ...), 'pred_overlay': Figure(id='5137', ...), 'rgb_input': tensor([[[109, 127, 151],
         [109, 127, 151],
         [109, 127, 151],
         ...,
         [101, 121, 146],
         [102, 122, 147],
         [103, 123, 148]],

        [[107, 125, 149],
         [107, 125, 149],
         [107, 125, 149],
         ...,
         [101, 121, 146],
         [100, 120, 145],
         [100, 120, 145]],

        [[104, 122, 146],
         [104, 122, 146],
         [104, 122, 146],
         ...,
         [ 95, 112, 138],
         [ 95, 112, 138],
         [ 95, 112, 138]],

        ...,

        [[ 62,  67,  73],
         [ 63,  68,  74],
         [ 63,  68,  74],
         ...,
         [ 45,  49,  58],
         [ 45,  49,  58],
         [ 45,  49,  58]],

        [[ 64,  67,  74],
         [ 64,  67,  74],
         [ 63,  66,  73],
         ...,
         [ 45,  49,  58],
         [ 45,  49,  58],
         [ 45,  49,  58

In [ ]:
# Save selected figures

save_ids = [0, 3]
for idx in save_ids:
    save_image_figure(all_figures[idx], f'images/examples_{idx}.jpg')